In [15]:
from sympy import *

In [16]:
t = Symbol('t') 
x = symbols('x', cls=Function)
h = symbols('h', cls=Function)
J = symbols('J', cls=Function)
C1, C2, A1, A2 = symbols('C1 C2 A1 A2')
t1, t2, x1, x2, l, T = symbols('t1 t2 x1 x2 l T')

In [17]:
inp = ''
inp
flag = 0
a = ''
b = ''
t11 = ''
t12 = ''
print("Введите лагранжиан: ")
inp = input()
print("Введите терминальную часть, если она есть, иначе -: ")
inp1 = input()
print("Введите параметры t1, t2: ")
t11 = input("t1: ")
t12 = input("t2: ")
if inp1 == '-':
    print("Введите параметры x(t1), x(t2). Если конец не закреплен, то поставьте -: ")
    a = input("x(t1): ")
    b = input("x(t2): ")
    if a == '-':
        if b == '-':
            flag = 12
        else:
            flag = 1
    elif b == '-':
        flag = 2
else:
    flag = 3

Введите лагранжиан: 
(x(t).diff(t))**2-x(t)
Введите терминальную часть, если она есть, иначе -: 
x2**2 + x1**2
Введите параметры t1, t2: 
t1: 0
t2: 1


In [18]:
#вариационная задача
#lagr = t**(Rational(2,3)) * (x(t).diff(t))**2
#lagr1 = "x(t)*t + (x(t).diff(t))**2"
#lagr2 = "(x(t).diff(t))**2-x(t)"
#l = x2**2 + x1**2
lagr = parse_expr(inp)
t1 = parse_expr(t11)
t2 = parse_expr(t12)
#t1 = 0
#t2 = 1
#x1 = 0
#x2 = 0
if inp1 != '-':
    l = parse_expr(inp1)
Eq(J(t),Integral(lagr,(t,t1,t2))+l)

Eq(J(t), x1**2 + x2**2 + Integral(-x(t) + Derivative(x(t), t)**2, (t, 0, 1)))

In [19]:
#уравнение Эйлера
f = (-lagr.diff(x(t).diff(t))).diff(t) + lagr.diff(x(t))
feq = Eq(f,0)
feq

Eq(-2*Derivative(x(t), (t, 2)) - 1, 0)

In [20]:
#решение уравнения Эйлера
sol = dsolve(feq,x(t)).simplify()
dsol = sol.rhs.diff(t)
rsol = sol.rhs
sol

Eq(x(t), C1 + C2*t - t**2/4)

In [21]:
#вычисляем константы
if flag == 0:
    x1 = parse_expr(a)
    x2 = parse_expr(b)
    sol1 = sol.subs(t,t1)
    sol1 = sol1.subs(x(t1),x1)

    sol2 = sol.subs(t,t2)
    sol2 = sol2.subs(x(t2),x2)
    
elif flag == 1:
    #L_x'(t1)=0
    x2 = parse_expr(b)
    sol1 = Eq(lagr.diff(x(t).diff(t)).subs(x(t).diff(t), dsol).subs(t,t1), 0)
    
    sol2 = sol.subs(t,t2)
    sol2 = sol2.subs(x(t2),x2)
    
elif flag == 2:
    #L_x'(t2)=0
    x1 = parse_expr(a)
    sol1 = sol.subs(t,t1)
    sol1 = sol1.subs(x(t1),x1)
    
    sol2 = Eq(lagr.diff(x(t).diff(t)).subs(x(t).diff(t), dsol).subs(t,t2), 0)
    
elif flag == 12:
    sol1 = Eq(lagr.diff(x(t).diff(t)).subs(x(t).diff(t), dsol).subs(t,t1), 0)
    sol2 = Eq(lagr.diff(x(t).diff(t)).subs(x(t).diff(t), dsol).subs(t,t2), 0)
    
elif flag == 3:
    #L_x'(t1) = l_(x'(t1))
    L1 = lagr.diff(x(t).diff(t))
    sol1 = Eq(L1.subs(x(t).diff(t),dsol).subs(t,t1),l.diff(x1).subs(x1,rsol).subs(t,t1))
    sol2 = Eq(L1.subs(x(t).diff(t),dsol).subs(t,t2),l.diff(x2).subs(x2,rsol).subs(t,t2))
    

In [22]:
sol1

Eq(2*C2, 2*C1)

In [23]:
sol2

Eq(2*C2 - 1, 2*C1 + 2*C2 - 1/2)

In [24]:
a = linsolve([sol1,sol2],(C1,C2))
A1 = list(a)[0][0]
A2 = list(a)[0][1]
A1, A2

(-1/4, -1/4)

In [25]:
#допустимая экстремаль
extr = (sol.subs(C1,A1)).subs(C2,A2)
extr

Eq(x(t), -t**2/4 - t/4 - 1/4)

In [26]:
#Варьирование
dlagr = (lagr.subs(x(t),x(t)+h(t)).simplify() - lagr).expand()
Eq((J(t+h(t))-J(t)).simplify(),dlagr)

Eq(-J(t) + J(t + h(t)), -h(t) + Derivative(h(t), t)**2 + 2*Derivative(h(t), t)*Derivative(x(t), t))

In [27]:
#упрощение
#Из необходимого условия экстремума L_x * h + L_x' * h' = 0
dlagr = dlagr - (lagr.diff(x(t)) * h(t) + lagr.diff(x(t).diff(t)) * h(t).diff(t))
dlagr


Derivative(h(t), t)**2

In [28]:
if (degree(dlagr, h(t).diff(t)) % 2 == 0) and (degree(dlagr, x(t).diff(t)) % 2 == 0):
    dlagr = dlagr.subs(h(t).diff(t),t).subs(x(t).diff(t),t)
    if (degree(dlagr, h(t)) % 2 == 0) and (degree(dlagr, x(t)) % 2 == 0):
        dlagr = dlagr.subs(h(t),t).subs(x(t),t)
        print(solve_univariate_inequality(dlagr.subs(h(t).diff(t),t) >= 0, t) and (t1 < t, t < t2))
    else:
        print(False)
else:
    print(False)

(0 < t, t < 1)
